# KOMO-3: Manipulation Modelling & Execution

The discussed components (KOMO, BotOp, NLP_Solver, RRT) provide basic ingredients for manipulation planning and execution. This tutorial is about how to practically use these in typical manipulation settings.

The first focus is on *manipulation modelling*. While KOMO provides a very powerful abstract framework to define all kinds of constraints, here we discuss what are concrete useful constraints for typical actions, e.g., picking and placing a box, or capsule. The *ManipulationModelling* class is meant to translate between typical actions and the abstract KOMO specification of the corresponding constraints.

The second focus is on the whole pipeline. We follow a basic sense-plan-act pipeline (not yet a fully integrated reactive framework such as SecMPC). To be more detailed, we assume the following basic steps in each loop:
* Perception: Update the configuration model to be in sync with the real world - using perception.
* Discrete decisions (task planning): Decide on discrete next actions, such as which object to pick or place next.
* Waypoint planning: Model the manipulation constraints for the next few actions and solve them to get a plan for the next few waypoints.
* Path planning: Create a fine-grained path/trajectory between waypoints, sometimes justing quick interpolation & optimization, sometimes using full fledge path finding (bi-directional RRT).
* Execution: Sending the path to BotOp for running it on the real system.

We neglect perception and discrete decision making here.

## Manipulation Modelling

We start with discussing manipulation modelling for standard box/cylinder grasping and placing.

In [1]:
import robotic as ry
import numpy as np
import random
import time

# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip

A basic configuration with a box and cylinder:

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.06,.06,.005]) \
    .setColor([1,.5,0]) \
    .setContact(1)

C.addFrame("capsule") \
    .setShape(ry.ST.capsule, [.2,.02]) \
    .setPosition([.25,.1,1.]) \
    .setColor([1,.5,0]) \
    .setContact(1)

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box = "box"
table = "table"
boxSize = C.frame(box).getSize()

C.view()

0

Look into the definition of *ManipulationModelling* class! You see that this class provides wrapper methods to setup a komo problem. The following demonstrate the methods provided to model box and cylinder grasping IK problems:

### Box centered top grasp
There are 6 possible orientation of an orthonormal centered box grasp. Have a look at the `grasp_top_box` method!

In [3]:
C.setJointState(qHome)
for orientation in ['xy', 'xz', 'yx', 'yz', 'zx', 'zy']: #loops over the 6 possible grasps
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    man.grasp_top_box(1., gripper, box, orientation)
    
    # solve it
    pose = man.solve()
    
    # check feasibility and display
    if man.ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_top_box with orientation {orientation}\nret: {man.ret}')
    else:
        print(' -- infeasible')

  -- feasible:
     { time: 0.009253, evals: 45, done: 1, feasible: 1, sos: 0.121124, f: 0, ineq: 0, eq: 0.0594381 }
  -- feasible:
     { time: 0.003294, evals: 18, done: 1, feasible: 1, sos: 0.0353606, f: 0, ineq: 0, eq: 0.00981419 }
  -- feasible:
     { time: 0.004489, evals: 22, done: 1, feasible: 1, sos: 0.0508421, f: 0, ineq: 0, eq: 0.00981408 }
  -- feasible:
     { time: 0.005633, evals: 12, done: 1, feasible: 1, sos: 0.00573579, f: 0, ineq: 0, eq: 0.00982528 }
  -- feasible:
     { time: 0.004201, evals: 15, done: 1, feasible: 1, sos: 0.0401536, f: 0, ineq: 0, eq: 0.00981435 }
  -- feasible:
     { time: 0.002628, evals: 13, done: 1, feasible: 1, sos: 0.0279681, f: 0, ineq: 0, eq: 0.00979114 }


By comparing the sum of squares of each of the calculated IK solutions, you can choose the most optimal/easy grasping position:

In [4]:
def find_best_grasp_orientation():
    best_orientation = ""
    best_orientation_sos = np.inf
    for orientation in ['xy', 'xz', 'yx', 'yz', 'zx', 'zy']:
        man = manip.ManipulationModelling(C)
        man.setup_inverse_kinematics()
        man.grasp_top_box(1., gripper, box, orientation)
        man.solve(verbose=0)
        
        if man.feasible and man.ret.sos < best_orientation_sos:
            best_orientation_sos = man.ret.sos
            best_orientation = orientation
    return best_orientation

C.setJointState(qHome)
best_orientation = find_best_grasp_orientation()
if best_orientation:
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    man.grasp_top_box(1., gripper, box, best_orientation)
    pose = man.solve()

    C.setJointState(pose[0])
    C.view(True, f'grasp_top_box with orientation {orientation}\nret: {man.ret}')

else:
    print("No feasible grasp found!")

  -- feasible:
     { time: 0.002003, evals: 7, done: 1, feasible: 1, sos: 0.0057358, f: 0, ineq: 0, eq: 0.00984279 }


### Box general grasp
We do not have to grasp a box in the center or orthonormally. We can only specify along which axis the fingers should press, and that they need to be inside a margin of the box sides.  Have a look at the `grasp_box` method! To illustrate the gained degrees of freedom, we also impose a random bias (leading to different solutions in nullspace):

In [5]:
C.setJointState(qHome)
limits = C.getJointLimits()
for orientation in ['x', 'y', 'z']:
    for i in range(10):
        # setup the manipulation problem
        man = manip.ManipulationModelling(C)
        man.setup_inverse_kinematics()
        # ... with random bias in joint space
        qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
        man.bias(1., qBias, 1e0)
        # ... and general, non-centered box grasping
        man.grasp_box(1., gripper, box, palm, orientation, margin=.02)
        
        # solve
        pose = man.solve()
        
        # if feasible, display
        if man.ret.feasible:
            C.setJointState(pose[0])
            C.view(True, f'grasp_box with orientation {orientation}\nret: {man.ret}')
        else:
            print('-- infeasible', i, orientation)

  -- feasible:
     { time: 0.013126, evals: 85, done: 1, feasible: 1, sos: 5.41142, f: 0, ineq: 8.44512e-06, eq: 5.96446e-05 }
  -- feasible:
     { time: 0.009705, evals: 27, done: 1, feasible: 1, sos: 3.45618, f: 0, ineq: 0.000232149, eq: 0.00284113 }
  -- feasible:
     { time: 0.038059, evals: 201, done: 1, feasible: 1, sos: 18.8161, f: 0, ineq: 0.0157777, eq: 0.0106554 }
  -- feasible:
     { time: 0.019184, evals: 81, done: 1, feasible: 1, sos: 5.24244, f: 0, ineq: 4.39661e-06, eq: 3.0533e-05 }
  -- feasible:
     { time: 0.036037, evals: 201, done: 1, feasible: 1, sos: 19.1286, f: 0, ineq: 0.000106372, eq: 0.000143786 }
  -- feasible:
     { time: 0.01369, evals: 48, done: 1, feasible: 1, sos: 3.34525, f: 0, ineq: 4.53839e-07, eq: 3.78107e-06 }
  -- feasible:
     { time: 0.019494, evals: 113, done: 1, feasible: 1, sos: 5.77332, f: 0, ineq: 0, eq: 0.00940403 }
  -- feasible:
     { time: 0.009057, evals: 43, done: 1, feasible: 1, sos: 2.68608, f: 0, ineq: 0, eq: 7.55489e-07 }
 

### Cylinder grasp

A cylinder (or capsule) can be grasped by ensuring the finger axis is normal to the cylinder's axis -- have a look at the `grasp_cylinder` method. Again, a demo with random bias to show the variety of grasps modelled that way:

In [6]:
C.setJointState(qHome)
limits = C.getJointLimits()
for i in range(10):
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
    man.bias(1., qBias, 1e0)
    man.grasp_cylinder(1., gripper, 'capsule', palm)
    
    # solve
    pose = man.solve()
    
    # if feasible, display
    if man.ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_cylinder\nret: {man.ret}')
    else:
        print('-- infeasible', i, orientation)

  -- feasible:
     { time: 0.007607, evals: 54, done: 1, feasible: 1, sos: 4.54864, f: 0, ineq: 0, eq: 1.29525e-06 }
  -- feasible:
     { time: 0.039914, evals: 201, done: 1, feasible: 1, sos: 12.7204, f: 0, ineq: 0, eq: 0.0100022 }
  -- feasible:
     { time: 0.025059, evals: 179, done: 1, feasible: 1, sos: 25.7362, f: 0, ineq: 0, eq: 4.2676e-06 }
  -- feasible:
     { time: 0.013641, evals: 64, done: 1, feasible: 1, sos: 4.38086, f: 0, ineq: 1.72203e-07, eq: 6.09746e-07 }
  -- infeasible:
     { time: 0.029838, evals: 200, done: 1, feasible: 0, sos: 12.7154, f: 0, ineq: 0.454625, eq: 2.59808 }
-- infeasible 4 z
  -- feasible:
     { time: 0.015301, evals: 114, done: 1, feasible: 1, sos: 20.0428, f: 0, ineq: 0, eq: 2.3597e-06 }
  -- feasible:
     { time: 0.031093, evals: 198, done: 1, feasible: 1, sos: 9.06975, f: 0, ineq: 0.074866, eq: 0.0933141 }
  -- feasible:
     { time: 0.024081, evals: 163, done: 1, feasible: 1, sos: 7.96976, f: 0, ineq: 0.0020079, eq: 0.0190007 }
  -- feasi

## Sequential Manipulation Modelling

Sequential manipulation modelling is special, as in some phases the manipulated objects move with the manipulator. Internally, komo models this with a *mode switch* (where an object becomes attached to the manipulator with a stable (but optimizable) relative transform).

Using the ManipulationModelling class, the `setup_pick_and_place_waypoints` method creates a two-time-steps komo problem where the relative object-gripper position is constrained to be the same in the 1st and 2st step (as it is parameterized by a shared relative kinematic joint). The `grasp_box` method ensures that the solution *also* fulfils grasp constraints in the first time step; and the `place_box` method ensures that the solution *also* fulfils placement constraints in the second time step. The additional `target_relative_xy_position` is optional, so see placement to explicit xy-positions on the table. Have a look at the definitions of all these methods.

In [7]:
C.setJointState(qHome)

for i in range(10):
    grasp_ori = random.choice(['x', 'y', 'z'])
    place_ori = 'z' #random.choice(['x', 'y', 'z'])
    info = f'pnp {i}, grasp orientation {grasp_ori}, place orientation {place_ori}'
    print('===', info)
    
    # setup manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_pick_and_place_waypoints(gripper, box)
    man.grasp_box(1., gripper, box, palm, grasp_ori)
    man.place_box(2., box, table, palm, place_ori)
    man.target_relative_xy_position(2., box, table, [(i%5)*.1-.2, .2])
    
    # solve
    q = man.solve()

    # if feasible, display (including 'fake' simulation with kinematic attach)
    if man.ret.feasible:
        C.setJointState(q[0])
        C.view(True, f'{info}\nwaypoint 0\nret: {man.ret}')
        C.attach(gripper, box)
        C.setJointState(q[1])
        C.view(True, f'{info}\nwaypoint 1\nret: {man.ret}')
        C.attach(table, box)
        C.setJointState(qHome)
        C.view(True, 'back home')
    else:
        print(' -- infeasible')

del man
C.frame('box').setPosition([-.25,.1,1.])
C.view()

=== pnp 0, grasp orientation x, place orientation z
  -- feasible:
     { time: 0.026945, evals: 54, done: 1, feasible: 1, sos: 0.0467499, f: 0, ineq: 2.83277e-08, eq: 1.87771e-06 }
=== pnp 1, grasp orientation x, place orientation z
  -- feasible:
     { time: 0.026306, evals: 59, done: 1, feasible: 1, sos: 0.0376289, f: 0, ineq: 0, eq: 0.00146225 }
=== pnp 2, grasp orientation x, place orientation z
  -- feasible:
     { time: 0.034216, evals: 48, done: 1, feasible: 1, sos: 0.0331689, f: 0, ineq: 7.27721e-09, eq: 3.94499e-05 }
=== pnp 3, grasp orientation x, place orientation z
  -- feasible:
     { time: 0.04902, evals: 104, done: 1, feasible: 1, sos: 0.0384227, f: 0, ineq: 0, eq: 0.00386248 }
=== pnp 4, grasp orientation y, place orientation z
  -- feasible:
     { time: 0.018815, evals: 32, done: 1, feasible: 1, sos: 0.020272, f: 0, ineq: 1.4644e-08, eq: 8.51998e-05 }
=== pnp 5, grasp orientation z, place orientation z
  -- infeasible:
     { time: 0.045814, evals: 81, done: 1, fe

0

## Path generation

Once solutions to the manipulation keyframes/waypoints are available, the next step is to generate motion between them. We can use sample-based path finding (bi-directional RRT) and/or smooth motion optimization for this.

### Smooth point-to-point motion
The following demonstrates smooth point-to-point motion between box grasps, there the motion is additionally constrains the endeffector to retract and approach:

In [8]:
C.setJointState(qHome)
limits = C.getJointLimits()
verbose = 0

for i in range(20):
    qStart = C.getJointState()
    
    # choose a random grasp orientation
    orientation = random.choice(['x', 'y', 'z'])
    print('===', i, 'orientation', orientation)
    
    # setup the grasp problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics(accumulated_collisions=True)
    man.grasp_box(1., gripper, box, palm, orientation)
    
    # solve
    pose = man.solve()
    print('    IK:', man.ret)
    
    # if feasible, display; otherwise try another grasp
    if man.ret.feasible:
        if verbose>0:
            C.setJointState(pose[0])
            C.view(True, f'grasp {i} with orientation {orientation}\nret: {man.ret}')
    else:
        print('  -- infeasible')
        C.setJointState(qStart)
        continue

    # setup the motion problem
    man = manip.ManipulationModelling(C, helpers=[gripper])
    man.setup_point_to_point_motion(qStart, pose[0])
    man.retract([.0, .2], gripper)
    man.approach([.8, 1.], gripper)
    
    # solve
    path = man.solve()
    print('  path:', man.ret)

    # if feasible, display trivially (no real execution in BotOp here)
    if man.ret.feasible:
        for t in range(path.shape[0]):
            C.setJointState(path[t])
            C.view(False, f'grasp {i} with orientation {orientation}, path step {t}\n{man.ret}')
            time.sleep(.05)
        C.view(verbose>0, f'path done')
    else:
        print('  -- infeasible')
        
del man

=== 0 orientation z
  -- feasible:
     { time: 0.015842, evals: 71, done: 1, feasible: 1, sos: 0.0549882, f: 0, ineq: 1.03635e-05, eq: 2.47906e-05 }
    IK: { time: 0.015842, evals: 71, done: 1, feasible: 1, sos: 0.0549882, f: 0, ineq: 1.03635e-05, eq: 2.47906e-05 }
  -- feasible:
     { time: 0.060421, evals: 34, done: 1, feasible: 1, sos: 2.32325, f: 0, ineq: 7.92239e-07, eq: 4.76558e-05 }
  path: { time: 0.060421, evals: 34, done: 1, feasible: 1, sos: 2.32325, f: 0, ineq: 7.92239e-07, eq: 4.76558e-05 }
=== 1 orientation y
  -- feasible:
     { time: 0.011025, evals: 37, done: 1, feasible: 1, sos: 0.0124815, f: 0, ineq: 1.36218e-07, eq: 1.88186e-06 }
    IK: { time: 0.011025, evals: 37, done: 1, feasible: 1, sos: 0.0124815, f: 0, ineq: 1.36218e-07, eq: 1.88186e-06 }
  -- feasible:
     { time: 0.040583, evals: 18, done: 1, feasible: 1, sos: 1.38363, f: 0, ineq: 1.12311e-06, eq: 0.000170265 }
  path: { time: 0.040583, evals: 18, done: 1, feasible: 1, sos: 1.38363, f: 0, ineq: 1.12311

## Integrated Example

Let's start with an integrated example, where the robot endlessly loops through picking and placing a box on a table.

In [9]:
import robotic as ry
import manipulation as manip
import numpy as np
#from importlib import reload
import time
import random

We define a basic configuration with box on the table:

In [10]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.15,.06,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setContact(1) \
    .setMass(.1)

C.addFrame("obstacle", "table") \
    .setShape(ry.ST.ssBox, [.06,.15,.06,.005]) \
    .setColor([.1]) \
    .setRelativePosition([-.15,.3-.055,.095]) \
    .setContact(1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box = "box"
table = "table"
boxSize = C.frame(box).getSize()

C.view()

0

### tob grasps and pick-and-place over an object

In [12]:
#reload(manip)

C.setJointState(qHome)
C.view_raise()

C.frame(box).setRelativePosition([-.0,.3-.055,.095])
C.frame(box).setRelativeQuaternion([1.,0,0,0])

for i in range(7):
        qStart = C.getJointState()

        graspDirection = 'yz' #random.choice(['xz', 'yz'])
        placeDirection = 'z'
        place_position = [(i%3)*.3-.3, .2]
        place_orientation = [-(i%2),((i+1)%2),0.]
        info = f'placement {i}: grasp {graspDirection} place {placeDirection} place_pos {place_position} place_ori {place_orientation}'
        print('===', info)

        M = manip.ManipulationModelling(C, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, joint_limits=False)
        M.grasp_top_box(1., gripper, box, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.target_relative_xy_position(2., box, table, place_position)
        M.target_x_orientation(2., box, place_orientation)
        M.solve()
        if not M.feasible:
                continue

        M2 = M.sub_motion(0)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.ret.feasible:
            continue

        M3 = M.sub_motion(1)
#         M3.retract([.0, .2], box, distance=.05)
#         M3.approach([.8, 1.], box, distance=.05)
        M3.keep_distance([], table, box)
        M3.keep_distance([], box, 'obstacle')
        M3.bias(.5, qHome, 1e0)
        M3.solve()
        if not M3.ret.feasible:
            continue
            
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp yz place z place_pos [-0.3, 0.2] place_ori [0, 1, 0.0]
  -- feasible:
     { time: 0.005623, evals: 18, done: 1, feasible: 1, sos: 0.0739073, f: 0, ineq: 0, eq: 6.83801e-05 }
  -- feasible:sub_motion_0--
     { time: 0.050241, evals: 27, done: 1, feasible: 1, sos: 1.02374, f: 0, ineq: 5.77954e-07, eq: 5.96644e-05 }
  -- feasible:sub_motion_1--
     { time: 0.048353, evals: 21, done: 1, feasible: 1, sos: 1.54354, f: 0, ineq: 0.000492129, eq: 0.00363911 }
=== placement 1: grasp yz place z place_pos [0.0, 0.2] place_ori [-1, 0, 0.0]
  -- feasible:
     { time: 0.057882, evals: 201, done: 1, feasible: 1, sos: 0.240754, f: 0, ineq: 0, eq: 0.131469 }
  -- feasible:sub_motion_0--
     { time: 0.052615, evals: 29, done: 1, feasible: 1, sos: 0.97455, f: 0, ineq: 2.62499e-06, eq: 0.000460183 }
  -- feasible:sub_motion_1--
     { time: 0.072007, evals: 37, done: 1, feasible: 1, sos: 5.73389, f: 0, ineq: 0.00166826, eq: 0.00387684 }
=== placement 2: grasp yz place z place_po

### endless box pick and place with random pick and place orientations

In [14]:
#reload(manip)

C.delFrame('obstacle')

C.setJointState(qHome)
C.view_raise()

for l in range(20):
        qStart = C.getJointState()

        graspDirection = random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.keep_distance([], palm, table)
        M.target_relative_xy_position(2., box, table, [.2, .3])
        ways = M.solve()

        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.keep_distance([.3,.7], palm, box, margin=.05)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.keep_distance([], table, box)
        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

===-- WARNING:kin.cpp:getFrame:412(-1) cannot find frame named 'obstacle'
 placement 0: grasp y place zNeg
  -- infeasible:placement 0: grasp y place zNeg
     { time: 0.029359, evals: 45, done: 1, feasible: 0, sos: 0.117769, f: 0, ineq: 1.69837e-05, eq: 1.00006 }
=== placement 1: grasp z place y
  -- infeasible:placement 1: grasp z place y
     { time: 0.047369, evals: 100, done: 1, feasible: 0, sos: 0.217941, f: 0, ineq: 0.223824, eq: 2.92991 }
=== placement 2: grasp z place zNeg
  -- infeasible:placement 2: grasp z place zNeg
     { time: 0.026164, evals: 74, done: 1, feasible: 0, sos: 0.0859222, f: 0, ineq: 0.0657621, eq: 3.12881 }
=== placement 3: grasp z place z
  -- infeasible:placement 3: grasp z place z
     { time: 0.030542, evals: 75, done: 1, feasible: 0, sos: 0.162524, f: 0, ineq: 0.0141389, eq: 2.20762 }
=== placement 4: grasp z place yNeg
  -- infeasible:placement 4: grasp z place yNeg
     { time: 0.075109, evals: 201, done: 1, feasible: 0, sos: 0.278897, f: 0, ineq: 0.

### random pushes

In [ ]:
#from importlib import reload 
#reload(manip)

C.frame("l_panda_finger_joint1").setJointState(np.array([.0]))

obj = box
C.frame(obj).setRelativePosition([-.0,.3-.055,.095])
C.frame(obj).setRelativeQuaternion([1.,0,0,0])

for i in range(30):
     qStart = C.getJointState()

     info = f'push {i}'
     print('===', info)

     M = manip.ManipulationModelling(C, info, ['l_gripper'])
     M.setup_pick_and_place_waypoints(gripper, obj, 1e-1, accumulated_collisions=False)
     M.straight_push([1.,2.], obj, gripper, table)
     #random target position
     M.komo.addObjective([2.], ry.FS.position, [obj], ry.OT.eq, 1e1*np.array([[1,0,0],[0,1,0]]), .4*np.random.rand(3) - .2+np.array([.0,.3,.0]))
     M.solve()
     if not M.ret.feasible:
          continue

     M1 = M.sub_motion(0, accumulated_collisions=False)
     M1.retractPush([.0, .15], gripper, .03)
     M1.approachPush([.85, 1.], gripper, .03)
     M1.keep_distance([.15,.85], obj, "l_finger1", .02)
     M1.keep_distance([.15,.85], obj, "l_finger2", .02)
     M1.keep_distance([.15,.85], obj, 'l_palm', .02)
     M1.keep_distance([], table, "l_finger1", .0)
     M1.keep_distance([], table, "l_finger2", .0)
     M1.solve()
     if not M1.ret.feasible:
          continue

     M2 = M.sub_motion(1, accumulated_collisions=False)
     M2.komo.addObjective([], ry.FS.positionRel, [gripper, '_push_start'], ry.OT.eq, 1e1*np.array([[1,0,0],[0,0,1]]))
     M2.solve()
     if not M2.ret.feasible:
          continue

     M1.play(C, 1.)
     C.attach(gripper, obj)
     M2.play(C, 1.)
     C.attach(table, obj)

del M

## TODOS:
* Proper execution: BotOp instead of display with C
* RRTs
* additional planar motion constraint for in-plane manipulation
* more typical manipulation constraints: camera_look_at, push_straight, 

In [ ]:
del C